##### Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd
import numpy as np
import inspect
import os
import decimal

Cambiamos el path a data.

In [2]:
owd='/home/jupyter-felduran@alumnos.u-3a791/mif-2020'
#owd='/home/jupyter-fgaldames@alumnos.-18bbe/mif-2020'
os.chdir(owd)
cwd = os.getcwd()
cwd

PermissionError: [Errno 13] Permission denied: '/home/jupyter-felduran@alumnos.u-3a791/mif-2020'

In [3]:
relpath=cwd+'/data'
os.chdir(relpath)

In [4]:
cwd = os.getcwd()
cwd

'/home/jupyter-felduran@alumnos.u-3a791/mif-2020/data'

Hemos logrado cambiar el path de working directory.

Se cargan los datos para la tarea.

In [3]:
pv = Qcf.PresentValue()

In [4]:
swaps = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')

swaps=swaps[::-1].reset_index()
swaps = swaps.rename(columns={' BENCHMARK NAME ':'a', 'RATE\n(PERCENT)':'TASA'})
swaps.TASA = swaps.TASA / 100
auxiliar = swaps.drop(['index','a'], axis=1)
swaps = swaps.drop(['index','a'], axis=1)
swaps

,DATE,TASA
0,2019-10-01,0.0188
1,2019-10-02,0.0185
2,2019-10-03,0.0184
3,2019-10-04,0.0182
4,2019-10-07,0.0183
...,...,...
265,2020-10-22,0.0007
266,2020-10-23,0.0008
267,2020-10-26,0.0009
268,2020-10-27,0.0009


In [5]:
auxiliar

,DATE,TASA
0,2019-10-01,0.0188
1,2019-10-02,0.0185
2,2019-10-03,0.0184
3,2019-10-04,0.0182
4,2019-10-07,0.0183
...,...,...
265,2020-10-22,0.0007
266,2020-10-23,0.0008
267,2020-10-26,0.0009
268,2020-10-27,0.0009


In [6]:
algo=auxiliar.iloc[0:3,:]
print(algo)
print(algo['TASA'].product())

         DATE    TASA
0  2019-10-01  0.0188
1  2019-10-02  0.0185
2  2019-10-03  0.0184
6.39952e-06


In [7]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [8]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno)) 
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [9]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


Procedemos a hacer un objeto time_series de fixings.

In [10]:
fixings = Qcf.time_series()
nrows = auxiliar.shape[0]
#print(nrows)
ncols = auxiliar.shape[1]
#print(ncols)
for i in range(int(0),int(nrows)):
    #print(i)
    fechaauxiliar = Qcf.build_qcdate_from_string(auxiliar.loc[i]['DATE'])
    tasaauxiliar = auxiliar.loc[i]['TASA']
    fixings[fechaauxiliar] = tasaauxiliar
print('--- This is the end ---')

--- This is the end ---


Hemos realizado la serie de tiempo con todas los factores de interes hasta el momento.

Ahora transformamos un dataframe que posea todos sus elementos de forma Qcf.

In [11]:
dir(fecha1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getinitargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__instance_size__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__safe_for_unpickling__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_days',
 'add_months',
 'day',
 'day_diff',
 'description',
 'month',
 'month_diff_day_remainder',
 'set_day',
 'set_month',
 'set_year',
 'week_day',
 'year']

In [12]:
# Initial_Date = auxiliar['DATE'].apply(lambda DATE: Qcf.build_qcdate_from_string(DATE))
# print(Initial_Date)

In [13]:
# Initial_Date = Qcf.time_series_dates(fixings)
# print(Initial_Date)
# type(Initial_Date)

In [14]:
calendario = get_cal(BusCal.NY)
type(calendario)
dir(calendario)
for holiday in calendario.get_holidays():
    print(holiday)

1-1-2020
12-10-2020
1-1-2021
15-2-2021
11-10-2021
1-1-2022
10-10-2022
1-1-2023
9-10-2023
1-1-2024
14-10-2024
1-1-2025
13-10-2025
1-1-2026
12-10-2026
1-1-2027
11-10-2027
1-1-2028
9-10-2028
1-1-2029
8-10-2029
1-1-2030
14-10-2030
1-1-2031
13-10-2031
1-1-2032
11-10-2032
1-1-2033
10-10-2033
1-1-2034
9-10-2034
1-1-2035
8-10-2035
1-1-2036
13-10-2036
1-1-2037
12-10-2037
1-1-2038
11-10-2038
1-1-2039
10-10-2039
1-1-2040
8-10-2040
14-10-2041
13-10-2042
12-10-2043
10-10-2044
9-10-2045
8-10-2046
14-10-2047
12-10-2048
11-10-2049
10-10-2050
9-10-2051
14-10-2052
13-10-2053
12-10-2054
11-10-2055
9-10-2056
8-10-2057
14-10-2058
13-10-2059
11-10-2060
10-10-2061
9-10-2062
8-10-2063
13-10-2064
12-10-2065
11-10-2066
10-10-2067
8-10-2068
14-10-2069
13-10-2070


In [15]:
# prueba = calendario.next_busy_day(fecha1)
# print(prueba)

In [16]:
# auxiliarframe = { 'Initial_Date': Initial_Date }
# auxiliarresult = pd.DataFrame(auxiliarframe)
# print(auxiliarresult)

In [17]:
# auxiliarresult['Final_Date'] = auxiliarresult['Initial_Date'].apply(
#     lambda Initial_Date: calendario.next_busy_day(Initial_Date.add_days(1)))
# Initial_Date = auxiliar['DATE'].apply(lambda DATE: Qcf.build_qcdate_from_string(DATE))
# print(Initial_Date)
# print(auxiliarresult)
# print(auxiliarresult.loc[257])

In [18]:
# auxiliarresult.insert(2,'Tasa',Qcf.time_series_values(fixings))
# print(auxiliarresult)
# print(auxiliarresult.loc[257]['Tasa'])
# type(auxiliarresult.loc[257]['Tasa'])

In [19]:
def get_wf(fecha_inicial: Qcf.QCDate, fecha_final: Qcf.QCDate, valor_tasa: float) -> float:
    """
    Retorna el factor de capitalización para una tasa entre dos fechas.
    Supone que la convención de la tasa es LinAct360.
    """
    qc_fec_ini = fecha_inicial
    
    qc_fec_fin = fecha_final
    
    qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    return qc_tasa.wf(qc_fec_ini, qc_fec_fin)

In [20]:
# a = 225
# print(auxiliarresult.loc[a]['Initial_Date'])
# print(auxiliarresult.loc[a]['Final_Date'])
# print(auxiliarresult.loc[a]['Tasa'])
# auxauxaux = get_wf(auxiliarresult.loc[a]['Initial_Date'],auxiliarresult.loc[a]['Final_Date'],auxiliarresult.loc[a]['Tasa'])
# print(decimal.Decimal.from_float(auxauxaux))

In [21]:
# auxiliarresult['wf'] = auxiliarresult.apply(
#     lambda row: get_wf(row['Initial_Date'], row['Final_Date'], row['Tasa']),
#     axis=1
# )
# print(auxiliarresult)

In [22]:
# print(auxiliarresult.loc[0]['wf'])

In [23]:
# fechaaccprueba = Qcf.QCDate(5, 10, 2019)
# print(fechaaccprueba)
# nfechaaccprueba = calendario.next_busy_day(fechaaccprueba)
# print(nfechaaccprueba)

In [24]:
# indice=auxiliarresult['Final_Date'].loc[lambda Final_Date: Final_Date==nfechaaccprueba].index.values
# print(indice)

In [ ]:
# iaux = 1
# indiceinitial=int(auxiliarresult['Initial_Date'].loc[lambda Initial_Date: Initial_Date==auxiliarresult.loc[0]['Initial_Date']].index.values)
# indicefinal=indice=int(auxiliarresult['Final_Date'].loc[lambda Final_Date: Final_Date==nfechaaccprueba].index.values)
# for i in range(indiceinitial,indicefinal+1):
    
#     if(iaux==1):
#         prodaux = auxiliarresult.loc[i]['wf']
    
#     else:
#         prodaux = prodaux * auxiliarresult.loc[i]['wf']
    
#     iaux = iaux+1
    
    
# print('--- This is the end ---')
# print(prodaux)
# print('--- This is the end ---')   

## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

Insertamos las funciones que necesitaremos luego para las otras funciones

In [ ]:
fecha_inicial=Qcf.QCDate(14, 9, 2018)
fecha_final=Qcf.QCDate(29, 9, 2018)

fecha_inicial.day_diff(fecha_final)

In [ ]:
# @dataclass # syntactic sugar
# class OisCashflow:
#     start_date: Qcf.QCDate
#     end_date: Qcf.QCDate
#     settlement_date: Qcf.QCDate
#     notional: float
#     currency: Qcf.QCCurrency
#     amortization: float
#     amort_is_cashflow: bool
#     interest_rate: Qcf.QCInterestRate
#     on_index: Qcf.InterestRateIndex
#     spread: float
#     gearing: float
        
#     def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
#         """
#         Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
#         se calcula como:
        
#         (P - 1) * 360 / (accrual_date - self.start_date)
        
#         donde P es el producto de los factores de capitalización de todas las tasas overnight
#         entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
#         en la variable `fixings`.
        
#         Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
#         de tipo Qcf.time_series.
#         """
        
#         fecha_inicial = self.start_date
#         fecha_final = accrual_date
#         P= fixings.wf(self.start_date,accrual_date)
        
#         teq= ((P - 1) * 360)/(fecha_inicial.day_diff(fecha_final))
        
#         return teq
    
#     def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
#         """
#         Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
#         que se calcula con el método anterior.
        
#         Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
#         Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
#         de tipo Qcf.time_series.
#         """
#         pass
    
#     def amount(self, fixings: Qcf.time_series) -> float:
#         """
#         Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
#         Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
#         Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
#         de tipo Qcf.time_series.
#         """
#         pass
    
#     def test(self):
#         return 'test'
    
#     def test1(self, x: float):
#         if x == 0:
#             raise ValueError('x debe ser != 0')
#         return 1 / x
    
#     def test2(self, x: float):
#         return self.test1(x) + 1
    
#     def test3(self, x: float):
#         return x + self.notional

In [27]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        
        
        fecha_inicial = self.start_date
        fecha_final = calendario.next_busy_day(accrual_date)
        
        Initial_Date = Qcf.time_series_dates(fixings)
        
        auxiliarframe = { 'Initial_Date': Initial_Date }
        auxiliarresult = pd.DataFrame(auxiliarframe)
        
        auxiliarresult['Final_Date'] = auxiliarresult['Initial_Date'].apply(
        lambda Initial_Date: calendario.next_busy_day(Initial_Date.add_days(1)))
        
        auxiliarresult.insert(2,'Tasa',Qcf.time_series_values(fixings))
        
        auxiliarresult['wf'] = auxiliarresult.apply(
        lambda row: get_wf(row['Initial_Date'], row['Final_Date'], row['Tasa']),
        axis=1
        )
    
        indiceinitial=int(auxiliarresult['Initial_Date'].loc[lambda Initial_Date: Initial_Date==fecha_inicial].index.values)
        indicefinal=indice=int(auxiliarresult['Final_Date'].loc[lambda Final_Date: Final_Date==fecha_final].index.values)
        
        
        otherauxiliar = auxiliarresult.iloc[indiceinitial:indicefinal+1,:]
        
        P = otherauxiliar['wf'].product()
        
        TasaEquivalente = ((P - 1) * 360)/(fecha_inicial.day_diff(fecha_final))
        
        return TasaEquivalente
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
            
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        fecha_inicial = self.start_date
        fecha_final = calendario.next_busy_day(accrual_date)
        
        TasaEquivalente=self.get_accrued_rate(fecha_final,fixings)
        Intereses=self.notional * TasaEquivalente* (fecha_inicial.day_diff(fecha_final)/360)

        return Intereses 
    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        fecha_final=calendario.next_busy_day(self.end_date)
        InteresDevengado=self.get_accrued_interest(fecha_final,fixings)
        FlujoFinal=InteresDevengado+self.amortization
        return FlujoFinal

Construcción de un objeto `Qcf.InterestRateIndex`.

In [28]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [29]:
ois = OisCashflow(
    #Qcf.QCDate(13, 1, 2020),
    Qcf.QCDate(1, 10, 2019),
    #Qcf.QCDate(13, 1, 2021),
    Qcf.QCDate(28, 10, 2020),
    #Qcf.QCDate(13, 1, 2021),
    Qcf.QCDate(28, 10, 2020), 
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

Probamos las funciones aca abajo esperando que retornen un valor respectivo a su programación con tal de no tener un error fundamental, sino poder estar equivocados parcialmente.

Prueba de ois.get_accrued_rate(Qcf.QCDate()).

In [30]:
#a = ois.get_accrued_rate(Qcf.QCDate(20, 1, 2020),fixings)
#a = ois.get_accrued_rate(Qcf.QCDate(5, 10, 2019),fixings)
a = ois.get_accrued_rate(ois.end_date,fixings)
print(decimal.Decimal.from_float(a))

0.006933722465227534249854191017448101774789392948150634765625


Prueba de ois.get_accrued_interest(Qcf.QCDate(20,1,2020)).

In [31]:
#b=ois.get_accrued_interest(Qcf.QCDate(20, 1, 2020),fixings)
#b=ois.get_accrued_interest(Qcf.QCDate(5, 10, 2019),fixings)
b=ois.get_accrued_interest(ois.end_date,fixings)
print(decimal.Decimal.from_float(b))

75693.1369120672461576759815216064453125


In [32]:
#print(calendario.next_busy_day(Qcf.QCDate(28, 10, 2020)))

Prueba de ois.amount().

In [33]:
c=ois.amount(fixings)
print(decimal.Decimal.from_float(c))

1075693.13691206718795001506805419921875


In [34]:
fixings

In [35]:
type(fixings)

finrisk.QC_Financial_3.time_series

In [36]:
(Qcf.QCDate(1, 10, 2019)).day_diff(Qcf.QCDate(7, 10, 2019))

6

In [37]:
try:
    ois.test1(0)
except Exception as e:
    print('Hacer algo')

Hacer algo


In [38]:
print(ois.start_date)

1-10-2019


In [39]:
print(ois.notional)

10000000


## Funciones

In [40]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    return TasaEsperada # ¿? ¿de dónde sale esta variable? Notar que explícitamente dice que la función
                        # retorna None

In [42]:
def present_value(val_date: Qcf.QCDate, ois_cashflow: OisCashflow, zcc: Qcf.ZeroCouponCurve) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
    
    fwd = Qcf.ForwardRates()
    fwd.set_rates_icp_clp_leg(val_date, 1.0, ois_cashflow, zcc) # faltaba , después de cashflow
    vp_flot = vp.pv(val_date, ois_cashflow, zcc)
    return vp_flot

## Tests

In [43]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [44]:
amount_tol = 10000
rate_tol = .001

In [45]:
exitos = 0

In [46]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [48]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [49]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [50]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [51]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [52]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [53]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.00991314%


Se verificará el cálculo usando `df_fixings`.

In [54]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [55]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [56]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [57]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [58]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [59]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [60]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [61]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [62]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [63]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [64]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [65]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.00000000%


In [66]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [67]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 72,377.11


In [68]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [69]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [70]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,075,027.05


In [71]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [72]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: -0.000000


In [73]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [74]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [75]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [76]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [77]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [80]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
te = set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'TasaEsperada' is not defined

In [82]:
print(te)

NameError: name 'te' is not defined

In [83]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'TasaEsperada' is not defined

In [84]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'TasaEsperada' is not defined

In [85]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'TasaEsperada' is not defined

In [86]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'TasaEsperada' is not defined

### `present_value`

In [87]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'TasaEsperada' is not defined

In [88]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'TasaEsperada' is not defined

In [89]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'TasaEsperada' is not defined

In [90]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'TasaEsperada' is not defined

In [91]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'TasaEsperada' is not defined

In [92]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 4.5
